
# Regulations.gov Scrape Explorer

Use this notebook to inspect the latest Regulations.gov harvest stored in `data/app_data/ai_corpus.db`. It summarizes per-docket counts and lets you read sample government updates or public comments pulled by the pipeline.


In [11]:

from pathlib import Path
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
sns.set_context('talk')


In [13]:

DB_PATH = Path('../data/app_data/ai_corpus.db')
BLOB_DIR = Path('../data/comments/blobs')
assert DB_PATH.exists(), f"Database not found: {DB_PATH}"
connection = sqlite3.connect(DB_PATH)


In [14]:

query = """
SELECT
    collection_id,
    doc_id,
    title,
    submitter_name,
    submitter_type,
    submitted_at,
    text_path,
    pdf_path,
    COALESCE(json_extract(raw_meta, '$.document_kind'), 'response') AS document_kind
FROM documents
WHERE source = 'regulations_gov'
"""
reg_docs = pd.read_sql_query(query, connection)
reg_docs['document_kind'] = reg_docs['document_kind'].fillna('response')
reg_docs.head()


,collection_id,doc_id,title,submitter_name,submitter_type,submitted_at,text_path,pdf_path,document_kind
0,NTIA-2023-0005,NTIA-2023-0005-1453,Questions,None,None,2023-06-27T04:00:00Z,data/comments/blobs/a6/a63723c0a901bc77b465320...,data/NTIA-2023-0005/raw/NTIA-2023-0005-1453_at...,response
1,NTIA-2023-0005,NTIA-2023-0005-1451,NTIA COMMENTARY AI ACCOUNTABILITY ACEMOGLU JO...,None,None,2023-06-16T04:00:00Z,data/comments/blobs/1b/1b4888cec04e828f3e065d6...,data/NTIA-2023-0005/raw/NTIA-2023-0005-1451_at...,response
2,NTIA-2023-0005,NTIA-2023-0005-1139,Comment on FR Doc # 2023-07776,None,None,2023-06-15T04:00:00Z,data/comments/blobs/3a/3a26fd25977a528e94602c7...,data/NTIA-2023-0005/raw/NTIA-2023-0005-1139_at...,response
3,NTIA-2023-0005,NTIA-2023-0005-1092,Comment on FR Doc # 2023-07776,None,None,2023-06-15T04:00:00Z,data/comments/blobs/d9/d97b79a18462f41ae99d8f9...,data/NTIA-2023-0005/raw/NTIA-2023-0005-1092_at...,response
4,NTIA-2023-0005,NTIA-2023-0005-1121,Comment on FR Doc # 2023-07776,None,None,2023-06-15T04:00:00Z,data/comments/blobs/03/03b9515ba3dced6bf388862...,data/NTIA-2023-0005/raw/NTIA-2023-0005-1121_at...,response



## Counts per docket

Government-issued updates map to `document_kind = 'call'` (notices, supporting materials, etc.), while `response` rows represent public submissions. The table below aggregates both counts per docket.


In [15]:

counts = (
    reg_docs
    .groupby(['collection_id', 'document_kind'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={'call': 'government_updates', 'response': 'comments'})
    .reset_index()
)
counts['total_docs'] = counts['government_updates'] + counts['comments']
counts.sort_values('total_docs', ascending=False).head(10)


document_kind,collection_id,government_updates,comments,total_docs
43,NTIA-2023-0005,1,1451,1452
3,EPA-HQ-OW-2017-0300,0,746,746
44,NTIA-2023-0009,1,334,335
46,OMB-2023-0020,1,196,197
4,EPA-HQ-OW-2023-0551,0,70,70
16,FDA-2019-N-4750,0,13,13
2,EPA-HQ-OPPT-2015-0436,0,11,11
8,FAA-2015-3344,1,4,5
5,EPA-R02-OAR-2023-0175,0,3,3
6,EPA-R09-OAR-2023-0203,0,2,2



## Explore a specific docket

Set `DOCKET_ID` to any collection from the summary table to list government updates, comments, and sample text. Comment text is loaded from blob files written during extraction.


In [18]:

DOCKET_ID = 'EPA-HQ-OW-2017-0300'  # change as needed

subset = reg_docs[reg_docs['collection_id'] == DOCKET_ID].copy()
if subset.empty:
    raise ValueError(f"No rows for docket {DOCKET_ID}")
subset_counts = (subset['document_kind'].value_counts().rename_axis('document_kind').reset_index(name='count'))
subset_counts


,document_kind,count
0,response,746


In [19]:

gov_updates = subset[subset['document_kind'] == 'call'][['doc_id', 'title', 'submitted_at']].sort_values('submitted_at')
gov_updates.head(10)


,doc_id,title,submitted_at


In [20]:

comments = subset[subset['document_kind'] == 'response'][['doc_id', 'submitter_name', 'submitter_type', 'submitted_at', 'text_path']]
comments.head(10)


,doc_id,submitter_name,submitter_type,submitted_at,text_path
2126,EPA-HQ-OW-2017-0300-1914,None,None,2023-10-13T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2127,EPA-HQ-OW-2017-0300-1913,None,None,2023-10-13T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2128,EPA-HQ-OW-2017-0300-1912,None,None,2023-08-18T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2129,EPA-HQ-OW-2017-0300-1888,None,None,2021-05-20T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2130,EPA-HQ-OW-2017-0300-1889,None,None,2021-05-20T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2131,EPA-HQ-OW-2017-0300-1887,None,None,2021-04-21T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2132,EPA-HQ-OW-2017-0300-1871,None,None,2021-04-14T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2133,EPA-HQ-OW-2017-0300-1872,None,None,2021-04-14T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2134,EPA-HQ-OW-2017-0300-1882,None,None,2021-04-14T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...
2135,EPA-HQ-OW-2017-0300-1883,None,None,2021-04-14T04:00:00Z,/Users/spangher/Projects/stanford-research/rfi...


In [23]:

from IPython.display import Markdown

def read_text(sample_row, max_chars=1200):
    path = sample_row.get('text_path')
    if not path:
        return "<no text path recorded>"
    file_path = Path(path)
    if not file_path.is_absolute():
        file_path = BLOB_DIR / file_path.name
    if not file_path.exists():
        return f"<missing text file: {file_path}>"
    text = file_path.read_text(encoding='utf-8', errors='replace')
    return text[:max_chars] + ('
…' if len(text) > max_chars else '')

sample_size = min(5, len(comments))
sample = comments.sample(sample_size, random_state=42) if sample_size else pd.DataFrame()

for _, row in sample.iterrows():
    header = f"**{row['doc_id']} — {row['submitter_name'] or 'Unknown submitter'} ({row['submitted_at'] or 'no date'})**"
    body = read_text(row)
    display(Markdown(header + '\n' + body.replace('', '  ')))


SyntaxError: unterminated string literal (detected at line 13) (4279316321.py, line 13)


### Inspect a specific document

Set `DOC_ID` to a single comment or government update to view its metadata and text.


In [24]:

DOC_ID = None  # e.g., 'EPA-HQ-OW-2017-0300-0895'
if DOC_ID:
    doc_row = subset[subset['doc_id'] == DOC_ID]
    if doc_row.empty:
        raise ValueError(f"Doc {DOC_ID} not found in docket {DOCKET_ID}")
    display(doc_row)
    text = read_text(doc_row.iloc[0]) if doc_row.iloc[0]['document_kind'] == 'response' else '<government document stored as PDF>'
    print(text)
else:
    print('Set DOC_ID to inspect a specific record.')


Set DOC_ID to inspect a specific record.



### Save summary

Optionally persist the per-docket summary to CSV.


In [25]:

output_csv = Path('notebooks/regulations_gov_docket_summary.csv')
summary.to_csv(output_csv, index=False)
output_csv


OSError: Cannot save file into a non-existent directory: 'notebooks'